<a href="https://colab.research.google.com/github/adigeak/facial_recog/blob/master/voilajonnes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import cv2
import numpy as np
import scipy as sp
import math
from sklearn.feature_selection import SelectPercentile,f_classif

In [0]:
def integral_image(image): # we have an image as an Numpy array
    '''
    This function to make the integral image which is the clever way 
    of storing images as we have to find the summation of all the
    pixel again and again as this task is require high computation
    power. To easy it little bit we use the integral image.
    '''
    print(image.shape)
    print(len(image))
    print(len(image[0]))
    ii = np.zeros(image.shape)
    s = np.zeros(image.shape)
    for y in range(len(image)):
        for x in range(len(image[y])): # doubt
            s[y][x]= s[y-1][x] + image[y][x] if y-1 >= 0 else image[y][x]
            ii[y][x] = ii[y][x-1] + s[y][x] if x-1 >= 0 else s[y][x]
    return ii

In [0]:
class RectangeleRegion:
    def __init__(self,x,y,width,height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def compute_feature(self, ii):
        return ii[self.y + self.height][self.x + self.widht] + ii[self.y][self.x] - (ii[self.y+self.height][self.x]+ii[self.y][self.x+self.width])
        

In [0]:
# i = np.ones((4,5))
# print(integral_image(i))

In [0]:
class ViolaJones:
    def __init__(self, T = 10):
        """
        This function is used to define the number of weak classifer that
        the algorithm is going to use to create strong classifier using 
        adaboost.
        """
        self.T = T
        self.alphas = []
        self.clfs = []

    def train(self, training, pos_num, neg_num):
        training_data = []
        weights = np.zeros(len(training))
        for x in range(len(training)):
            traning_data.append((integral_image(training[x][0]),training[x][1])) # [x][1] is the classification of
            # Image is positive 1 and negative 0
            # if positive use 1/2*p where p is the number of positive classes and same for negative
            if training[x][1] == 1:
                weights[x] = 1.0 / (2 * pos_num)
            else:
                weights[x] = 1.0 / (2 * neg_num)
        features = self.build_features(training_data[0][0].shape)
        X, y = self.apply_features(features,training_data)
        indices = SelectPercentile(f_classif, percentile=10).fit(X.T, y).get_support(indices=True)
        X = X[indices]
        features = features[indices]
        for t in range(self.T):
            weights = weights / np.linalg.norm(weights) # normalize wight
            weak_classifiers = self.train_weak(X, y, features, weights)
            clf, error, accuracy = self.select_best(weak_classifiers, weights, training_data)
            beta = error / (1.0 - error)
            for i in range(len(accuracy)):
                weights[i] = weights[i] * (beta ** (1 - accuracy[i]))
                alpha = math.log(1.0/beta)
                self.alphas.append(alpha)
                self.clfs.append(clf)

    
    def train_weak(self, X, y, features, weights):
        total_pos, total_neg = 0, 0
        for w, label in zip(weights, y):
            if label == 1:
                total_pos += w
            else:
                total_neg += w

        classifiers = []
        total_features = X.shape[0]
        for index, feature in enumerate(X):
            if len(classifiers) % 1000 == 0 and len(classifiers) != 0:
                print("Trained %d classifiers out of %d" % (len(classifiers), total_features))
            applied_feature = sorted(zip(weights, feature, y), key=lambda x: x[1])
            pos_seen, neg_seen = 0, 0
            pos_weights, neg_weights = 0, 0
            min_error, best_feature, best_threshold, best_polarity = float('inf'), None, None, None
            for w, f, label in applied_feature:
                error = min(neg_weights + total_pos - pos_weights, pos_weights + total_neg - neg_weights)
                if error < min_error:
                    min_error = error
                    best_feature = features[index]
                    best_threshold = f
                    best_polarity = 1 if pos_seen > neg_seen else -1
                if label == 1:
                    pos_seen += 1
                    pos_weights += w
                else:
                    neg_seen += 1
                    neg_weights += w
            clf = WeakClassifier(best_feature[0], best_feature[1], best_threshold, best_polarity)
            classifiers.append(clf)
        return classifiers

    def select_best(self, classifiers, weights, trainig_data):
        best_clf, best_error, best_accuracy = None, float('inf'), None
        for clf in classifiers:
            error, accuracy = 0, []
            for data, w in zip(training_data, weights):
                correctness = abs(clf.classify(data[0]) - data[1])
                accuracy.append(correctness)
                error += w * correctness
            error = error / len(training_data)
            if error < best_error:
                best_clf, best_error, best_accuracy = clf, error, accuracy
        return best_clf, best_error, best_accuracy

    def build_features(self, image_shape):
        height, width = image_shape
        features = []
        for w in range(1,width+1):
            for h in range(1,height+1):
                i = 0
                while i + w < width:
                    j = 0
                    while j + h < height:
                        #2 rectangle features
                        immediate = RectangleRegion(i, j, w, h)
                        right = RectangleRegion(i+w, j, w, h)
                        if i + 2 * w < width: #Horizontally Adjacent
                            features.append(([right], [immediate]))

                        bottom = RectangleRegion(i, j+h, w, h)
                        if j + 2 * h < height: #Vertically Adjacent
                            features.append(([immediate], [bottom]))
                        
                        right_2 = RectangleRegion(i+2*w, j, w, h)
                        #3 rectangle features
                        if i + 3 * w < width: #Horizontally Adjacent
                            features.append(([right], [right_2, immediate]))

                        bottom_2 = RectangleRegion(i, j+2*h, w, h)
                        if j + 3 * h < height: #Vertically Adjacent
                            features.append(([bottom], [bottom_2, immediate]))

                        #4 rectangle features
                        bottom_right = RectangleRegion(i+w, j+h, w, h)
                        if i + 2 * w < width and j + 2 * h < height:
                            features.append(([right, bottom], [immediate, bottom_right]))
                        # we have use 4 Haar feature. 2 with two rectangle, 3 rectalge features
                        # 4 rectangle feature
                        j += 1
                    i += 1
        
        return np.array(features) # we can modify here becoz it had added all features
    
    def apply_features(self, features, trainDf): # this is like selecting the best feature 
    # before training
        X = np.zeros((len(features),len(trainDf)))
        Y = np.array(map(lambda d: d[1], trainDf))
        i = 0
        for pos_reg, neg_reg in features:
            feature = lambda ii: sum([pos.compute_feature(ii) for pos in positive_regions]) - sum([neg.compute_feature(ii) for neg in negative_regions])
            X[i] = list(map(lambda d: feature(d[0]), trainDF))
            i += 1
        return X,Y
    def classify(self, image):
        total = 0
        ii = integral_image(image)
        for alpha,clf in zip(self.alphas, self.clfs):
            total += alpha * clf.classify(ii)
        return 1 if total >= 0.5 * sum(self.alphas) else 0

    def save(self, filename):
        with open(filename+".pkl", 'wb') as f:
            pickle.dump(self, f)
    
    @staticmethod
    def load(filename):
        with open(filename+".pkl", 'rb') as f:
            return pickle.load(f)

In [0]:
class WeakClassifier:
    def __init__(self, pos_reg, neg_reg, thres, polarity):
        self.pos_reg = pos_reg
        self.neg_reg = neg_reg
        self.thres = thres
        self.polarity = polarity

    def classify(self, x):
        feature = lambda ii: sum([pos.compute_feature(ii) for pos in positive_regions]) - sum([neg.compute_feature(ii) for neg in negative_regions])
        return 1 if self.polarity * feature(x) < self.polarity * self.thres else 0   

In [0]:
def train(t):
    with open("training.pkl", 'rb') as f:
        training = pickle.load(f)
    clf = ViolaJones(T=t)
    clf.train(training, 2429, 4548)
    evaluate(clf, training)
    clf.save(str(t))

def test(filename):
    with open("test.pkl", 'rb') as f:
        test = pickle.load(f)
    clf = ViolaJones.load(filename)
    evaluate(clf, test)

def evaluate(clf, data):
    correct = 0
    for x, y in data:
        correct += 1 if clf.classify(x) == y else 0
    print("Classified %d out of %d test examples" % (correct, len(data)))

In [12]:
train(10)

FileNotFoundError: ignored